In [ ]:
# 0. Variabel Kontrol untuk Mengecek Apakah Program Sudah Dijalankan
program_run = False

# 1. Mengimpor Library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.decomposition import PCA

# Fungsi utama untuk menjalankan program
def main():
    global program_run

    # Memeriksa apakah program sudah dijalankan sebelumnya
    if program_run:
        print("Program sudah dijalankan. Tidak bisa dijalankan ulang.")
        return

    # Tandai program sudah dijalankan
    program_run = True

    # 2. Memuat Dataset Iris
    url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv'
    column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']
    data = pd.read_csv(url, names=column_names)

    # 3. Menyiapkan Data
    X = data.iloc[:, :-1].values  # Fitur: sepal_length, sepal_width, petal_length, petal_width
    y = data.iloc[:, -1].values   # Target: class

    # Mengonversi label menjadi angka (encoding)
    class_mapping = {label: idx for idx, label in enumerate(np.unique(y))}
    y = np.array([class_mapping[label] for label in y])

    print("Nama Kelas:", list(class_mapping.keys()))

    # 4. Membagi Dataset Menjadi Data Latih dan Data Uji
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # 5. Standarisasi Data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # 6. Membuat dan Melatih Model SVM
    svm_model = SVC(kernel='linear', C=1.0, random_state=42)
    svm_model.fit(X_train, y_train)

    # 7. Evaluasi Model
    y_pred = svm_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"\nAkurasi Model: {accuracy:.2f}")
    print(classification_report(y_test, y_pred, target_names=list(class_mapping.keys())))

    # Visualisasi Matriks Kebingungan
    conf_matrix = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, cmap='Blues', xticklabels=list(class_mapping.keys()), yticklabels=list(class_mapping.keys()))
    plt.xlabel("Prediksi")
    plt.ylabel("Aktual")
    plt.title("Matriks Kebingungan SVM")
    plt.show()

    # 8. Visualisasi Data dengan PCA
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X)

    # Membuat model SVM OvO
    svm_setosa_vs_versicolor = SVC(kernel='linear', C=1.0, random_state=42)
    svm_setosa_vs_virginica = SVC(kernel='linear', C=1.0, random_state=42)
    svm_versicolor_vs_virginica = SVC(kernel='linear', C=1.0, random_state=42)

    # Menyiapkan data untuk setiap model OvO
    mask_setosa_vs_versicolor = (y == 0) | (y == 1)
    mask_setosa_vs_virginica = (y == 0) | (y == 2)
    mask_versicolor_vs_virginica = (y == 1) | (y == 2)

    # Melatih model OvO
    svm_setosa_vs_versicolor.fit(X_pca[mask_setosa_vs_versicolor], y[mask_setosa_vs_versicolor])
    svm_setosa_vs_virginica.fit(X_pca[mask_setosa_vs_virginica], y[mask_setosa_vs_virginica])
    svm_versicolor_vs_virginica.fit(X_pca[mask_versicolor_vs_virginica], y[mask_versicolor_vs_virginica])

    # 9. Visualisasi dengan Hyperplane
    plt.figure(figsize=(10, 6))

    # Plot titik dengan transparansi dan ukuran yang lebih besar
    for i, target_name in enumerate(class_mapping.keys()):
        plt.scatter(X_pca[y == i, 0], X_pca[y == i, 1], label=target_name, alpha=0.7, s=60)

    # Hyperplane Setosa vs Versicolor
    w = svm_setosa_vs_versicolor.coef_[0]
    a = -w[0] / w[1]
    xx = np.linspace(X_pca[:, 0].min() - 1, X_pca[:, 0].max() + 1)
    yy = a * xx - (svm_setosa_vs_versicolor.intercept_[0]) / w[1]
    plt.plot(xx, yy, 'r--', label='Hyperplane Setosa vs Versicolor')

    # Hyperplane Setosa vs Virginica
    w = svm_setosa_vs_virginica.coef_[0]
    a = -w[0] / w[1]
    yy = a * xx - (svm_setosa_vs_virginica.intercept_[0]) / w[1]
    plt.plot(xx, yy, 'g--', label='Hyperplane Setosa vs Virginica')

    # Hyperplane Versicolor vs Virginica
    w = svm_versicolor_vs_virginica.coef_[0]
    a = -w[0] / w[1]
    yy = a * xx - (svm_versicolor_vs_virginica.intercept_[0]) / w[1]
    plt.plot(xx, yy, 'b--', label='Hyperplane Versicolor vs Virginica')

    # Mengatur plot
    plt.xlabel("Komponen Utama 1")
    plt.ylabel("Komponen Utama 2")
    plt.title("Visualisasi Data Iris dengan PCA dan Hyperplane SVM OvO")
    plt.legend(loc='upper right')

    # Menambahkan grid dan membuat plot lebih jelas
    plt.grid(True)
    plt.show()

# Memanggil fungsi main
main()
